### Chart of the Day

##### Food Inflation vs Wider Inflation

In [17]:
import pandas as pd, altair as alt, requests
import sys, importlib

sys.path.append('../')
import api_wrapper.api_hub as api_hub

- Dataset ID: MM23
- Series ID: 
    - `CPIH INDEX 01.1 : FOOD 2015=100` - L52H
    - `CPIH INDEX 00: ALL ITEMS 2015=100` - L522
    - `CPIH INDEX 01.1.1 : BREAD & CEREALS 2015=100` - L52I
    - `CPIH INDEX 01.1.2 : MEAT 2015=100` - L52J
    - `CPIH INDEX 01.1.3 : FISH 2015=100` - L52K
    - `CPIH INDEX 01.1.4 : MILK, CHEESE & EGGS 2015=100` - L52L

In [26]:
# Reload module & instantiate API wrapper
importlib.reload(api_hub)
api = api_hub.EconDataAPI()


series = {
    'id': ['L522', 'L52H', 'L52I', 'L52J', 'L52K', 'L52L'],
    'name': ['All items', 'Food', 'Bread and cereals', 'Meat', 'Fish', 'Milk, cheese and eggs']
}

# Get data and concatenate into single dataframe
df = pd.DataFrame()
for s in series['id']:
    df_temp = api.get_ons_data(dataset_id='MM23', series_id=s)
    df_temp['name'] = series['name'][series['id'].index(s)]
    df = pd.concat([df, df_temp], ignore_index=True)



In [22]:
# df_food = api.get_ons_data(dataset_id='MM23', series_id='L52H')
# df_all = api.get_ons_data(dataset_id='MM23', series_id='L522')

# df_food['series'] = 'Food'
# df_all['series'] = 'All'

# df = pd.concat([df_food, df_all], ignore_index=True)

In [27]:
df

,date,month,value,year,name
0,1988-01-01,January,46.9,1988,All items
1,1988-02-01,February,47.0,1988,All items
2,1988-03-01,March,47.2,1988,All items
3,1988-04-01,April,47.8,1988,All items
4,1988-05-01,May,48.0,1988,All items
...,...,...,...,...,...
2569,2023-05-01,May,143.3,2023,"Milk, cheese and eggs"
2570,2023-06-01,June,142.6,2023,"Milk, cheese and eggs"
2571,2023-07-01,July,142.1,2023,"Milk, cheese and eggs"
2572,2023-08-01,August,143.7,2023,"Milk, cheese and eggs"


---

### Chart

Time-series, lines for each series, indexed to 100 in 2015

In [32]:
import altair_wrapper.eco_styles as eco_styles
styles = eco_styles.EcoStyles()

In [45]:
# rename `name` column to `series`
df = df.rename(columns={'name': 'series'})

Let's calculate a rolling window average to smooth out the index.

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2574 entries, 0 to 2573
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2574 non-null   datetime64[ns]
 1   month   2574 non-null   object        
 2   value   2574 non-null   float64       
 3   year    2574 non-null   object        
 4   series  2574 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 100.7+ KB


In [61]:
# Convert 'date' to datetime format and set as index
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# Group by 'series' and calculate the 3-month rolling mean
rolling_means = df.groupby('series')['value'].rolling(window=3).mean().reset_index()

# Display the first few rows of the resulting dataframe
rolling_means.head()

,series,date,value
0,All items,1988-01-01,NaN
1,All items,1988-02-01,NaN
2,All items,1988-03-01,47.033333
3,All items,1988-04-01,47.333333
4,All items,1988-05-01,47.666667


In [134]:
# Calculate the rolling mean
rolling_mean = df.groupby('series')['value'].rolling(window=3).mean()

# Reset index to make 'series' and 'date' columns regular columns again for merging
rolling_mean = rolling_mean.reset_index()

# Merge this rolling mean with the original dataframe
df_merge = pd.merge(df.reset_index(), rolling_mean, on=['date', 'series'], suffixes=('', '_3m')).dropna()

# Find the rolling mean value for each series as of January 1, 2015
base_values = df_merge[df_merge['date'] == '2022-01-01'].set_index('series')['value_3m']

# Function to re-index the series
def reindex_series(row):
    base_value = base_values[row['series']]
    return 100 * row['value_3m'] / base_value if pd.notnull(base_value) else None

# Apply the re-indexing function to the dataframe
df_merge['value_3m'] = df_merge.apply(reindex_series, axis=1)

# Display the first few rows of the dataframe after re-indexing
df_merge.head()

,date,month,value,year,series,value_3m
2,1988-03-01,March,47.2,1988,All items,41.089109
3,1988-04-01,April,47.8,1988,All items,41.351194
4,1988-05-01,May,48.0,1988,All items,41.642400
5,1988-06-01,June,48.2,1988,All items,41.933605
6,1988-07-01,July,48.2,1988,All items,42.050087


In [167]:
# Print the values at the start of 2022 vs the most recent values
pd.concat([df_temp[df_temp['date'] == '2022-01-01'], df_temp[df_temp['date'] == '2023-09-01']])

,date,month,value,year,series,value_3m
408,2022-01-01,January,114.6,2022,All items,100.000000
1266,2022-01-01,January,108.3,2022,Bread and cereals,100.000000
1695,2022-01-01,January,103.1,2022,Meat,100.000000
2124,2022-01-01,January,114.3,2022,Fish,100.000000
2553,2022-01-01,January,104.8,2022,"Milk, cheese and eggs",100.000000
428,2023-09-01,September,130.1,2023,All items,113.133372
1286,2023-09-01,September,136.9,2023,Bread and cereals,126.343422
1715,2023-09-01,September,126.8,2023,Meat,123.842140
2144,2023-09-01,September,136.4,2023,Fish,121.686391
2573,2023-09-01,September,141.4,2023,"Milk, cheese and eggs",136.094298


In [176]:
# Edit names of series
series_names = {
    'All items': 'CPIH: All items',
}

# replace series names
df_merge['series'] = df_merge['series'].replace(series_names)

In [174]:
# Create colour scheme for chart
colours = {
    "All items": styles.eco_colours['blue-light'],
}

In [180]:
importlib.reload(eco_styles)
styles = eco_styles.EcoStyles()
styles.register_and_enable_theme(dark_mode=True)

# df_temp = df_merge.copy()
df_temp = df_merge[df_merge['series'] != 'Food'].copy()

domain = ['CPIH: All items', 'Bread and cereals', 'Fish', 'Meat', 'Milk, cheese and eggs']

title = alt.Title(
    text= 'UK food inflation',
    fontSize=14,
    subtitle='CPIH index 100 = 2022, 3-month rolling mean | Source: ONS'
)

chart = alt.Chart(df_temp, title=title).mark_line().encode(
    x=alt.X('date:T'),
    y=alt.Y('value_3m:Q', scale=alt.Scale(zero=False, padding=20)),
    color=alt.Color('series:N', legend=None, scale=alt.Scale(domain=domain)),
    strokeDash=alt.condition(alt.datum.series == 'CPIH: All items', alt.value([4,2]), alt.value([1,0]))
).transform_filter(
    alt.datum.year >= 2018
)

# add line at y=100
line = alt.Chart(pd.DataFrame({'y': [100]})).mark_rule(opacity=0.6, strokeWidth=2, yOffset=0.5).encode(y='y')


# Calculate last value for each series
last_values = df_temp.groupby('series').last().reset_index()

# Add points on to the end of the line
points = alt.Chart(last_values).mark_circle(
    size=45,
    opacity=1
).encode(
    x=alt.X('date:T'),
    y=alt.Y('value_3m:Q'),
    color=alt.Color('series:N'),
)

# Add text labels at the end of the line
text = alt.Chart(last_values).mark_text().encode(
    text=alt.Text('series:N'),
    x=alt.X('date:T'),
    y=alt.Y('value_3m:Q'),
    # color=alt.Color('series:N'),
).transform_filter(alt.datum.series != 'Fish').transform_filter(alt.datum.series != 'Bread and cereals')

text_bread = alt.Chart(last_values).mark_text(dy=-3).encode(
    text=alt.Text('series:N'),
    x=alt.X('date:T'),
    y=alt.Y('value_3m:Q'),
).transform_filter(alt.datum.series == 'Bread and cereals')

text_fish = alt.Chart(last_values).mark_text(dy=3).encode(
    text=alt.Text('series:N'),
    x=alt.X('date:T'),
    y=alt.Y('value_3m:Q'),
).transform_filter(alt.datum.series == 'Fish')


chart = line + chart + text + text_fish + text_bread + points
chart.display()

alt.LayerChart(...)

Save chart

In [181]:
styles.save(chart, '../../charts/202311/', '20231114_UK_Food_Inflation', width=360)